In [2]:
import os
import json
import numpy as np
from PIL import Image, ImageEnhance

# Paths to directories
specific_dates = ['20240718']         # use variable below when you have all geoJSON 
# specific_dates = ['20240718', '20230927', '20231108', '20240314', '20240516', '20240626']
base_tif_dir = './data/campaigns'
geojson_dir = './geojson_file'

output_dir_with_contrast = 'dataset/cropped_with_contrast'
output_dir_no_contrast = 'dataset/cropped_no_contrast'
output_label_dir = 'dataset/labels_cropped'

# Create output directories
os.makedirs(output_dir_with_contrast, exist_ok=True)
os.makedirs(output_dir_no_contrast, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

# Image dimensions for YOLO normalization
IMAGE_WIDTH = 1616
IMAGE_HEIGHT = 1240

def get_geojson_files(geojson_dir):
    """
    Get set of geoJSON file names without extensions.
    """
    return {os.path.splitext(f)[0] for f in os.listdir(geojson_dir) if f.endswith('.geojson')}

def create_yolo_annotation(file_path, class_id, x_center, y_center, width, height):
    """
    Save YOLO format annotation to a text file.
    """
    with open(file_path, 'w') as file:
        file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

def process_geojson_file(file_name, tif_dir):
    """
    Process a single geoJSON file and its corresponding .tif file.
    """
    tif_path = os.path.join(tif_dir, file_name + '.tif')
    geojson_path = os.path.join(geojson_dir, file_name + '.geojson')

    # Check if the corresponding `.tif` file exists
    if not os.path.exists(tif_path):
        print(f"Warning: `{tif_path}` not found. Skipping this file.")
        return

    # Load geoJSON
    with open(geojson_path, 'r') as file:
        geojson_data = json.load(file)

    # Open the TIFF file and process each shape in the GeoJSON
    with Image.open(tif_path) as tif_img:
        for feature in geojson_data['features']:
            process_feature(feature, tif_img, file_name)

def process_feature(feature, tif_img, file_name):
    """
    Process an individual feature within a geoJSON file.
    """
    # Class mapping
    class_mapping = {"Living": 0, "Non-Living": 1, "Bubble": 2}

    # Get classification
    classification = feature['properties']['classification']['name']
    class_id = class_mapping.get(classification, class_mapping["Non-Living"])

    # Get coordinates and frame index
    coordinates = feature['geometry']['coordinates'][0]
    frame_idx = feature['geometry']['plane']['t']

    # Access specified frame in the TIFF stack
    tif_img.seek(frame_idx)

    # Calculate bounding box from coordinates
    x_coords = [point[0] for point in coordinates]
    y_coords = [point[1] for point in coordinates]
    min_x, max_x = int(min(x_coords)), int(max(x_coords))
    min_y, max_y = int(min(y_coords)), int(max(y_coords))

    # Crop and save the shape images
    frame = np.array(tif_img)
    image_basename = save_cropped_images(frame, min_x, max_x, min_y, max_y, classification, file_name, frame_idx)

    # Calculate YOLO bounding box coordinates
    x_center = (min_x + max_x) / 2 / IMAGE_WIDTH
    y_center = (min_y + max_y) / 2 / IMAGE_HEIGHT
    width = (max_x - min_x) / IMAGE_WIDTH
    height = (max_y - min_y) / IMAGE_HEIGHT

    # Save YOLO annotations
    annotation_filename = image_basename + ".txt"
    annotation_path = os.path.join(output_label_dir, annotation_filename)
    create_yolo_annotation(annotation_path, class_id, x_center, y_center, width, height)

def save_cropped_images(frame, min_x, max_x, min_y, max_y, classification, file_name, frame_idx):
    """
    Save cropped shape images with and without contrast, and return the base filename.
    """
    # Crop the image
    shape_crop = frame[min_y:max_y, min_x:max_x]

    # Create base filename
    base_filename = f"{file_name}_{classification}_shape_{frame_idx}"

    # Save no-contrast image
    no_contrast_path = os.path.join(output_dir_no_contrast, base_filename + ".png")
    shape_image_no_contrast = Image.fromarray(shape_crop)
    shape_image_no_contrast.save(no_contrast_path)

    # Save contrast-enhanced image
    contrast_path = os.path.join(output_dir_with_contrast, base_filename + ".png")
    enhancer = ImageEnhance.Contrast(shape_image_no_contrast)
    shape_image_with_contrast = enhancer.enhance(30)  # Adjust contrast factor
    shape_image_with_contrast.save(contrast_path)

    return base_filename

def main():
    """
    Main function to process TIFF and GeoJSON files for all specified dates.
    """
    for date in specific_dates:
        tif_dir = os.path.join(base_tif_dir, date, 'image_stacks')
        print(f"Processing files for date: {date}")
        geojson_files = get_geojson_files(geojson_dir)
        for file_name in geojson_files:
            process_geojson_file(file_name, tif_dir)

if __name__ == "__main__":
    main()


Processing files for date: 20240718
